In [1]:
import os
import cv2

In [2]:
from keras.preprocessing import image

In [3]:
categories = ['with_mask', 'without_mask']

In [4]:
data=[]
for category in categories:
    path = os.path.join('train', category)
    
    label=categories.index(category)
    
for file in os.listdir(path):
    img_path = os.path.join(path,file)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    data.append([img,label])
    

In [5]:
len(data)

3828

In [6]:
import random

In [7]:
random.shuffle(data)

In [8]:
X=[]
y=[]
for features, label in data:
    X.append(features)
    y.append(label)

In [9]:
len(X)

3828

In [10]:
len(y)

3828

In [11]:
import numpy as np

In [12]:
X=np.array(X)
y=np.array(y)

In [13]:
X , y =X[:1000] , y[:1000]

In [14]:
X.shape

(1000, 224, 224, 3)

In [15]:
y.shape

(1000,)

In [16]:
X=X/255

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [18]:
X_train.shape

(800, 224, 224, 3)

In [19]:
from keras.applications.vgg16 import VGG16

In [20]:
vgg=VGG16()

In [21]:
from keras import Sequential

In [22]:
model=Sequential()

In [23]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [25]:
for layer in model.layers:
    layer.trainable=False

In [26]:
from keras.layers import Dense

In [27]:
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'] )

In [ ]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
25/25 [==============================] - 889s 35s/step - loss: 0.0311 - accuracy: 0.9887 - val_loss: 1.9133e-06 - val_accuracy: 1.0000
Epoch 2/5
 2/25 [=>............................] - ETA: 7:36 - loss: 2.2202e-06 - accuracy: 1.0000

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
    print(y_pred)
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF==ord('x'):
        break
cv2.destroyAllWindows()        

In [ ]:
def detect_face_mask(img):
    y_pred=model.predict_classes(img.reshape(1,224,224,3))
    return y_pred

In [ ]:
sample1=cv2.imread('sample/1.jpg')
sample1=cv2.resize(sample1,(224,224))

In [ ]:
detect_face_mask(sample1)